# EAT example #2: Variational

3D-Var biogeochemical data assimilation with the BFM biogeochemical model at a site in the Mediterranean Sea

## Import

In [ ]:
# Imports from standard library
import sys
import shutil
import warnings
warnings.filterwarnings("ignore")

# Import of third party packages
import numpy as np
%matplotlib widget
from matplotlib import pyplot
import cmocean

# Shared postprocessing scripts
sys.path.append("..")
import shared

## Load observations

In [ ]:
obs = shared.read_0d_observations("observations/nrt_chlsat.obs")

# Reference simulation

Forecast-only, no data assimilation

In [ ]:
%cd reference

In [ ]:
!eat-gotm

## Load and plot results

In [ ]:
# Load results
time, z, chl, _, _ = shared.read_result("result.nc", "total_chlorophyll")

# Create figure
fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(
    figsize=(8, 6), nrows=2, ncols=2, width_ratios=[0.95, 0.05], sharex="col"
)

# Plot surface chlorophyll
shared.plot_0d_timeseries(ax1, time, chl[..., -1], obs)
ax1.set_ylabel("chlorophyll (mg m⁻³)")
ax1.set_title("surface chlorophyll")
cax1.axis("off")

# Plot modelled chlorophyll throughout the water column
pc, cb = shared.plot_1d_timeseries(ax2, time, z, chl, cmap=cmocean.cm.algae, cax=cax2)
cb.set_label("chlorophyll (mg m⁻³)")
ax2.set_title("chlorophyll")
ax2.set_ylim(300.0, z.min())

fig.tight_layout()

fig.savefig("chl_ref.png", dpi=150)

# Data assimilation

In [ ]:
%cd ../da
shutil.copyfile("../reference/gotm.yaml", "gotm.yaml")
shutil.copyfile("../reference/fabm.yaml", "fabm.yaml")
!mpiexec -n 1 python run.py : -n 1 eat-gotm

## Load and plot results

### Chlorophyll

In [ ]:
# Load results
time_da, z_da, chl_da, _, _ = shared.read_result("result_0001.nc", "total_chlorophyll")

# Create figure
fig, ((ax1, cax1), (ax2, cax2), (ax3, cax3)) = pyplot.subplots(
    figsize=(8, 8),
    nrows=3,
    ncols=2,
    sharex="col",
    width_ratios=[0.95, 0.05],
)

# Plot surface chlorophyll
shared.plot_0d_timeseries(
    ax1,
    time_da,
    chl_da[:, -1],
    extra_series=[("model, no DA", chl[:, -1])],
    obs=obs,
    label="model, DA (3DVar)",
)
ax1.set_ylabel(f"chlorophyll (mg m⁻³)")
ax1.set_title("surface chlorophyll (a)")
cax1.axis("off")

# Plot original modelled chlorophyll throughout the water column [free run]
cf, cb = shared.plot_1d_timeseries(
    ax2,
    time,
    z,
    chl,
    np.linspace(0.0, 0.5, 11),
    cmap=cmocean.cm.algae,
    extend="max",
    cax=cax2,
)
cb.set_label("chlorophyll (mg m⁻³)")
ax2.set_title(f"chlorophyll without data assimilation (b)")
ax2.set_ylim(300.0, z.min())

# Plot change in chlorophyll due to data assimilation
chl_diff = chl_da - chl
contours = np.linspace(-0.18, 0.18, 19)
cf, cb = shared.plot_1d_timeseries(
    ax3,
    time_da,
    z_da,
    chl_diff,
    contours,
    cmap=cmocean.cm.balance,
    extend="both",
    cax=cax3,
)
cb.set_label("chlorophyll difference (mg m⁻³)")
ax3.set_title("impact of data assimilation on chlorophyll [DA (3DVar) - no DA] (c)")
ax3.set_ylim(300.0, z.min())

fig.tight_layout()

fig.savefig("chl_da.png", dpi=150)

### Primary production

In [ ]:
# Load results (aggregate production from 4 plankton functional types)
pp_ref = []
pp_da = []
for pft in range(1, 5):
    _, _, pp, _, _ = shared.read_result("../reference/result.nc", f"P{pft}_run")
    pp_ref.append(pp)
    _, _, pp, _, _ = shared.read_result("result_0001.nc", f"P{pft}_run")
    pp_da.append(pp)
pp_ref = np.sum(pp_ref, axis=0)
pp_da = np.sum(pp_da, axis=0)

# Create figure
fig, ((ax1, cax1), (ax2, cax2)) = pyplot.subplots(
    figsize=(8, 6),
    nrows=2,
    ncols=2,
    sharex="col",
    width_ratios=[0.95, 0.05],
)

# Plot original modelled primary production throughout the water column [free run]
cf, cb = shared.plot_1d_timeseries(
    ax1,
    time,
    z,
    pp_ref,
    20,
    extend="max",
    cax=cax1,
)
cb.set_label("primary production (mg m⁻³ d⁻¹)")
ax1.set_title(f"primary production without data assimilation (a)")
ax1.set_ylim(200.0, z.min())

# Plot change in chlorophyll due to data assimilation
pp_diff = pp_da - pp_ref
contours = np.linspace(-3, 3, 31)
cf, cb = shared.plot_1d_timeseries(
    ax2,
    time_da,
    z_da,
    pp_diff,
    contours,
    cmap=cmocean.cm.balance,
    extend="both",
    cax=cax2,
)
cb.set_label("pp difference (mg m⁻³ d⁻¹)")
ax2.set_title(
    "impact of data assimilation on primary production [DA (3DVar) - no DA] (b)"
)
ax2.set_ylim(200.0, z.min())

fig.tight_layout()

fig.savefig("pp_da.png", dpi=150)